In [3]:
import pandas as pd
import os
import socks
import socket
import urllib
import urllib2
import time
from bs4 import BeautifulSoup
from multiprocessing import Pool
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import stopit
import zlib

TIMEOUT = 1.0 # 1 second
N_PROC = 16
MAX_LINE_LENGTH = 4000

targets = map(lambda x : x.lower(),
              ["Trump", 
               "Cruz", 
               "Rubio", 
               "Carson",
               "Paul",
               "Bush",
               "Huckabee",
               "Kasich",
               "Fiorina",
               "Christie",
               "Santorum",
               "O'Malley",
               "Clinton", 
               "Sanders", 
               "ad", 
               "commercial"])

event_table_headers = ['GLOBALEVENTID',
                       'SQLDATE',
                       'MonthYear',
                       'Year',
                       'FractionDate',
                       'Actor1Code',
                       'Actor1Name',
                       'Actor1CountryCode',
                       'Actor1KnownGroupCode',
                       'Actor1EthnicCode',
                       'Actor1Religion1Code',
                       'Actor1Religion2Code',
                       'Actor1Type1Code',
                       'Actor1Type2Code',
                       'Actor1Type3Code',
                       'Actor2Code',
                       'Actor2Name',
                       'Actor2CountryCode',
                       'Actor2KnownGroupCode',
                       'Actor2EthnicCode',
                       'Actor2Religion1Code',
                       'Actor2Religion2Code',
                       'Actor2Type1Code',
                       'Actor2Type2Code',
                       'Actor2Type3Code',
                       'IsRootEvent',
                       'EventCode',
                       'EventBaseCode',
                       'EventRootCode',
                       'QuadClass',
                       'GoldsteinScale',
                       'NumMentions', 
                       'NumSources',
                       'NumArticles',
                       'AvgTone',
                       'Actor1Geo_Type',
                       'Actor1Geo_FullName',
                       'Actor1Geo_CountryCode',
                       'Actor1Geo_ADM1Code',
                       'Actor1Geo_Lat',
                       'Actor1Geo_Long',
                       'Actor1Geo_FeatureID',
                       'Actor2Geo_Type',
                       'Actor2Geo_FullName',
                       'Actor2Geo_CountryCode',
                       'Actor2Geo_ADM1Code',
                       'Actor2Geo_Lat',
                       'Actor2Geo_Long',
                       'Actor2Geo_FeatureID',
                       'ActionGeo_Type',
                       'ActionGeo_FullName',
                       'ActionGeo_CountryCode',
                       'ActionGeo_ADM1Code',
                       'ActionGeo_Lat',
                       'ActionGeo_Long',
                       'ActionGeo_FeatureID',
                       'DATEADDED',
                       'SOURCEURL']

In [4]:
def extract_freqs(targets, url):

    """
    def create_connection(address, timeout=None, source_address=None):
        sock = socks.socksocket()
        sock.connect(address)
        return sock

    socks.setdefaultproxy(socks.PROXY_TYPE_SOCKS5, "127.0.0.1", 9050)

    # patch the socket module
    socket.socket = socks.socksocket
    socket.create_connection = create_connection
    # ...
    """
    
    start_time = time.clock()
    try:
        
        with stopit.ThreadingTimeout(3) as to_ctx_mgr:
            assert to_ctx_mgr.state == to_ctx_mgr.EXECUTING
            
            try:
        
                user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
                values = {'name' : 'Micael Foord',
                          'location' : 'Northmpton',
                          'language' : 'Python' }
                headers = { 'User-Agent' : user_agent, 'Content-Type' : 'html/text' }

                data = urllib.urlencode(values)
                req = urllib2.Request(url, data, headers)
                html = urllib2.urlopen(req, timeout = TIMEOUT).read()
                soup = BeautifulSoup(html, "lxml")

                # kill all script and style elements
                for script in soup(["script", "style"]):
                    script.extract()    # rip it out

                # get text
                text = soup.get_text()

                # break into lines and remove leading and trailing space on each
                lines = (line.strip() for line in text.splitlines() )
                # break multi-headlines into a line each
                chunks = [zlib.compress(line.strip().encode('utf-8'))                        
                          for line in lines
                          if len(line) < MAX_LINE_LENGTH]
                # drop blank lines                                

                return chunks, time.clock() - start_time, None
            
            except Exception as e:
                return None, time.clock() - start_time, repr(e)
                
        return None, time.clock() - start_time, 'Exceeded timeout for processing'

    except (Exception, KeyboardInterrupt, SystemExit) as e:
        return None, time.clock() - start_time, repr(e)

def extract_freqs_(args):
    return extract_freqs(args[0], args[1])

In [5]:
INPUT_FOLDER = '../GDELT/'
OUTPUT_FOLDER = '../GDELT_grepped/'

stride = 2 * N_PROC

In [6]:
file_name = np.random.permutation(os.listdir(INPUT_FOLDER))[0]

event_table = pd.read_csv(INPUT_FOLDER + file_name, 
                          sep = '\t', 
                          header = None, 
                          names = event_table_headers)


/usr/local/lib/python2.7/dist-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,13,14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [ ]:
count_table = event_table.copy()
count_table['grabed'] = None

p = Pool(N_PROC)

from collections import Counter

letter_counts = Counter(event_table['SOURCEURL'].apply(lambda x : x.split('/')[:3][-1]))

targets = [y[0] for y in sorted(letter_counts.items(), key = (lambda x : - x[1]))[:50]]

print len(event_table)
event_table = event_table[event_table.apply(lambda x : x['SOURCEURL'].split('/')[:3][-1], axis = 1).isin(targets)]
print len(event_table)

123501
26454


In [ ]:
n_events = len(event_table)
for i in xrange((n_events / stride) + 1):
#for i in xrange(3):
    start_chunk = time.clock()
    print 'Processing ' + str(file_name) + ' chunk ' + str(i) + ' out of ' + str(n_events / stride + 1) 
    start = i * stride
    end = min((i + 1) * stride, n_events)
    chunk = event_table[start : end]
    print len(chunk)
    count_table['grabed'][start:end] = p.map(extract_freqs_, zip([targets] * len(chunk), chunk['SOURCEURL'].values))
    #count_table.to_csv(OUTPUT_FOLDER + 'counts_' + file_name)
    print time.clock() - start_chunk
print 'Done processing ' + str(file_name)
count_table.to_csv(OUTPUT_FOLDER + 'counts_' + file_name)

Processing 20130802.export.CSV chunk 0 out of 827
32
0.182215
Processing 20130802.export.CSV chunk 1 out of 827
32
0.041342
Processing 20130802.export.CSV chunk 2 out of 827
32
0.033011
Processing 20130802.export.CSV chunk 3 out of 827
32
0.033107
Processing 20130802.export.CSV chunk 4 out of 827
32
0.032188
Processing 20130802.export.CSV chunk 5 out of 827
32
0.03223
Processing 20130802.export.CSV chunk 6 out of 827
32
0.034006
Processing 20130802.export.CSV chunk 7 out of 827
32
0.035999
Processing 20130802.export.CSV chunk 8 out of 827
32
0.033318
Processing 20130802.export.CSV chunk 9 out of 827
32
0.031963
Processing 20130802.export.CSV chunk 10 out of 827
32
0.03259
Processing 20130802.export.CSV chunk 11 out of 827
32
0.032704
Processing 20130802.export.CSV chunk 12 out of 827
32
0.032402
Processing 20130802.export.CSV chunk 13 out of 827
32
0.042584
Processing 20130802.export.CSV chunk 14 out of 827
32
0.031933
Processing 20130802.export.CSV chunk 15 out of 827
32
0.03274
Proce

In [ ]:
print 'Done'

In [ ]:
print count_table